In [1]:
import pandas as pd
from unstructured.partition.pdf import partition_pdf
import re, json

In [2]:
DSM_code = pd.read_csv("./DSM_codes.txt", sep="\t")

In [3]:
DSM_code.head()

,"Disorder, condition, or problem",ICD-9–CM,ICD-10–CM
0,Academic or educational problem,V62.3,Z55.9
1,Acculturation difficulty,V62.4,Z60.3
2,Acute stress disorder,308.3,F43.0
3,Adjustment disorder,NaN,NaN
4,"Adjustment disorder, Unspecified",309.9,F43.20


In [6]:
DSM_code.to_csv("DSM_codes.csv", index=False)

In [2]:
filename = "./sample.pdf"

elements = partition_pdf(filename=filename, infer_table_structure=True)
tables = [el for el in elements if el.category == "Table"]

# print(tables[0].text)
# print(tables[0].metadata.text_as_html)

[nltk_data] Downloading package punkt to /Users/anselm/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/anselm/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
def extract_dsm_criteria(filename):
    elements = partition_pdf(filename=filename)
    dsm_5 = dict()

    for i, element in enumerate(elements):
        if element.text == "Diagnostic Criteria":
            # Get the next element
            # print(i)
            prev_element = elements[i - 1]

            # Print the text
            disease = prev_element.text

            j = i + 1
            criteria = []
            code = None
            while ("Subtypes" not in elements[j].text) and ("Diagnostic Features" not in elements[j].text):
                if re.match(r"^F\d+.?\d+$", elements[j].text):
                    code = elements[j].text
                elif not re.match(r"^\d+$", elements[j].text):
                    criteria.append(elements[j].text)
                j += 1
            criteria_text = "\n".join(criteria)
            dsm_5[disease] = {"code": code, "criteria": criteria_text}

    json.dump(dsm_5, open("dsm_5.json", "w"))
    return dsm_5

In [3]:
dsm_5 = extract_dsm_criteria("./DSM-5-TR.pdf")

In [4]:
import json

dsm_5 = json.load(open("./dsm_5.json"))

In [5]:
len(dsm_5.keys())

158

In [20]:
dsm_5["SECTION II"]

{'code': None,
 'criteria': 'Neurodevelopmental Disorders\nSchizophrenia Spectrum and Other Psychotic Disorders\nBipolar and Related Disorders\nDepressive Disorders\nAnxiety Disorders\nObsessive-Compulsive and Related Disorders\nTrauma- and Stressor-Related Disorders\nDissociative Disorders\nSomatic Symptom and Related Disorders\nFeeding and Eating Disorders\nElimination Disorders\nSleep-Wake Disorders\nSexual Dysfunctions\nGender Dysphoria\nDisruptive, Impulse-Control, and Conduct Disorders\nSubstance-Related and Addictive Disorders\nNeurocognitive Disorders\nPersonality Disorders\nParaphilic Disorders\nOther Mental Disorders and Additional Codes\nMedication-Induced Movement Disorders and Other Adverse Effects of Medication\nOther Conditions That May Be a Focus of Clinical Attention\nThis section contains the diagnostic criteria approved for routine clinical use along with ICD-10-CM diagnostic codes. For each mental disorder, the diagnostic criteria are followed by descriptive text to

In [12]:
dsm_5["Delusional Disorder"]

{'code': None,
 'criteria': 'F22\nA. The presence of one (or more) delusions with a duration of 1 month or longer. B. Criterion A for schizophrenia has never been met.\nNote: Hallucinations, if present, are not prominent and are related to the delusional theme (e.g., the sensation of being infested with insects associated with delusions of infestation).\nC. Apart from the impact of the delusion(s) or its ramifications, functioning is not\nmarkedly impaired, and behavior is not obviously bizarre or odd.\nD. If manic or major depressive episodes have occurred, these have been brief\nrelative to the duration of the delusional periods.\nE. The disturbance is not attributable to the physiological effects of a substance or another medical condition and is not better explained by another mental disorder, such as body dysmorphic disorder or obsessive-compulsive disorder.\nSpecify whether:\nErotomanic type: This subtype applies when the central theme of the delusion is that another person is in

In [17]:
import re

re.match(r"^F\d+.?\d+$", "F22.3")

<re.Match object; span=(0, 5), match='F22.3'>